## Анализ рынка продажи автомобильных ковриков в период 01.11.2021 - 16.02.2022

Данный анализ проводился с целью выявления самых популярных марок автомобилей на основе продаж автомобильных ковриков конкурентов, для проверки гипотезы и возможного дальнейшего заказа пробной партии для реализации на маркетплейсе. Цель анализа - узнать объемы рынка и определить популярные модели авто.

Импортируем необходимые для работы библиотеки

In [1]:
import pandas as pd
import os # для работы с файлами
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

С сайта shopstat.ru забираем необходимую статистику. Статистику можно скачать в формате .csv, по 50 записей в одном файле.

В данном проекте будем автоматически загружать необходимые файлы со статистикой. Период - 01/11/2021 - 16/02/2022". Данные взяты с маркетплейса **ozon.ru**

Вводим функцию для загрузки файлов. Было перепробовано много решений, взятых с интернета, заработало только решение ниже, в режиме эмуляции браузера. Причина - сайт отдает файл через несколько секунд после открытия ссылки. Если просто попытаться скачать файл по ссылке - ошибка 403 Forbidden. 

Код неидеален, работает медленно, но зато автоматически.

In [2]:
def headless_browser(url):
    # The following lines are so the browser is headless, i.e. it doesn't open a window
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=600x600')
    

    wd = webdriver.Chrome('1/chromedriver.exe')

    wd.get(url)  # Open the desired url in the browser

    time.sleep(3)
    wd.close()  # Close the browser

Вручную необходимо сделать следующее - сначала на сайте shopstat.ru заходим в нужную категорию, конфигурируем искомый период данных и получаем ссылку на первый csv файл. 

In [3]:
page_cnt = 1
url = 'https://new-api.shopstat.ru/export/product-list?marketplace=OZON&categoryId=8633&sortBy=SOLD&sortDirection=DESC&fromDate=1635714000000&toDate=1644958800000&page=' + str(page_cnt) + '&token=EcR-6fDt-KAMAx4GbMK_U'

Циклом загружаем необходимое количество файлов:

In [4]:
for i in range(100):
    headless_browser(url)
    page_cnt +=1

Файлы упали в папку загрузок по умолчанию. Переместим их в текущий каталог (или пропишем путь до туда, куда они упали) и объеденим. 

In [5]:
full_df = pd.read_csv('data\8633-1-export.csv', encoding='cp1251')

In [6]:
for i in range(1, 100):
    i += 1
    path = 'data\8633-' + str(i) + '-export.csv'
    temp_df = pd.read_csv(path, encoding='cp1251')
    full_df = pd.concat([full_df, temp_df])

Проверим количество строк, колонок, удалим дубликаты, если есть.

In [7]:
full_df = full_df.drop_duplicates()

In [8]:
full_df.shape

(5000, 14)

Посмотрим содержимое, первые три строки:

In [9]:
full_df.head(3)

,SKU,Название,Категория,Бренд,Остаток,Отзывов,Минимальная цена,Максимальная цена,Средняя цена,Продаж,Выручка,Ссылка на товар,Фото,Рейтинг
0,198852991,"Автовизитка на торпеду автомобиля, визитная ка...",Обустройство салона,NicePrice,3492,3827,816,816,196,9580,1851716,https://www.ozon.ru/context/detail/id/198852991/,https://cdn1.ozone.ru/s3/multimedia-f/62233123...,4.80
1,311489537,"Оплетка, чехол кпп, чехол ручника, мультиразме...",Обустройство салона,-,64,1,740,740,740,1954,1445960,https://www.ozon.ru/context/detail/id/311489537/,https://cdn1.ozone.ru/s3/multimedia-7/60925947...,4.00
2,164286489,"Коврики в салон Element, для Kia Rio, 06/2017-...",Обустройство салона,Element,646,11,1819,1819,1642,1566,2539862,https://www.ozon.ru/context/detail/id/164286489/,https://cdn1.ozone.ru/multimedia/1033679384.jpg,4.52


Очистим данные - уберем ненужные колонки, такие как - SKU, остаток, мин.цену, макс.цену, ссылки, фото и рейтинг

In [10]:
full_df.drop(['SKU', 'Категория', 'Остаток', 'Минимальная цена', 'Максимальная цена', 'Ссылка на товар', 'Фото', 'Рейтинг'], axis=1, inplace=True) 
full_df.head(3)

,Название,Бренд,Отзывов,Средняя цена,Продаж,Выручка
0,"Автовизитка на торпеду автомобиля, визитная ка...",NicePrice,3827,196,9580,1851716
1,"Оплетка, чехол кпп, чехол ручника, мультиразме...",-,1,740,1954,1445960
2,"Коврики в салон Element, для Kia Rio, 06/2017-...",Element,11,1642,1566,2539862


Переименуем оставшиеся колонки для удобства работы:

In [11]:
full_df = full_df.rename(columns = {'Средняя цена': 'mean_price', 'Продаж': 'total_sales', 
                                    'Выручка': 'total_money',  'Отзывов': 'comments',
                                   'Название': 'name',  'Бренд': 'brand'},)

Выведем в эксель для отправки топ 100 вещей, в категории "обустройство салона". Вдруг нас заинтересуют продажи чего-либо еще и мы захотим заняться данным товаром на маркетплейсе. 

In [12]:
full_df.sort_values('total_sales', ascending = False).head(100).reset_index() \
    .drop(['index'], axis=1).to_excel("top_100_items.xlsx")

Фильтруем ДФ по содержимому колонки name. 

In [13]:
carpets = full_df.loc[full_df['name'].str.contains("ковр|ковер|ковёр", case=False)]
carpets.sort_values('total_sales', ascending = False).reset_index() \
    .drop(['index'], axis=1)

,name,brand,comments,mean_price,total_sales,total_money
0,Автомобильный коврик (впитывает влагу и грязь)...,3TON,152,134,1720,230153
1,"Коврики в салон Element, для Kia Rio, 06/2017-...",Element,11,1642,1566,2539862
2,"Коврик автомобильный Верона С, влаговпитывающи...",Верона-С,83,178,1148,205757
3,Коврики в салон автомобиля Rival для Hyundai S...,Rival,4,1496,974,1453489
4,"Коврики влаговпитывающие DolleX 50 х 38 см, 2 шт",DolleX,103,177,938,166256
...,...,...,...,...,...,...
1414,Коврик на торпеду с логотипом KIA,-,9,303,6,1756
1415,Автомобильные EVA коврики с бортами на Lada Ve...,AUTORET,1,3000,6,18000
1416,Коврик на приборную панель с логотипом Lada Лада,Haomai,18,281,6,1725
1417,Резиновые коврики Сетка Seintex для BMW 3 Ser ...,Seintex,0,3502,6,21540


Выводим в эксель, на этом моменте анализ окончен.

In [14]:
carpets.sort_values('total_sales', ascending = False).reset_index() \
    .drop(['index'], axis=1).to_excel("all_carpets.xlsx")